In [38]:
import pandas as pd
import requests
import json
import networkx as nx

In [39]:
csv = pd.read_csv("MediaCloud/biomass-all-story-urls-20210422214554.csv")

csv.tail(1)


,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
7177,1908931813,2021-04-21 06:45:00,Why the federal government is buying into the ...,https://grist.org/energy/can-the-federal-gover...,en,False,"environment, energy and power, air pollution, ...",27343,Grist,http://grist.org/


In [43]:

# Working ID 333682533

G = nx.Graph()

id = str(1908931813)

url = 'https://api.mediacloud.org/api/v2/stories_public/single/' + id + '?key=b2ef1a99a8fdbb84afafe742fd437c0942703072bb98242ba0eed9b8411e1735'
r = requests.get(url, allow_redirects=True)

y = json.loads(r.content)

# y

orgs = []

for t in y[0]['story_tags']:
    if (t['tag_set'] == 'cliff_organizations'):
        orgs.append(t)
        # print(t['tag'])
        # print(t['tags_id'])
        # print()

orgs

# open(id +'.json', 'wb').write(r.content)



[{'stories_id': 1908931813,
  'tag': 'Google',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9363209},
 {'stories_id': 1908931813,
  'tag': 'Department of Defense',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9366963},
 {'stories_id': 1908931813,
  'tag': 'tech giants',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9376410},
 {'stories_id': 1908931813,
  'tag': 'GSA',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9398493},
 {'stories_id': 1908931813,
  'tag': 'clean energy',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9584691},
 {'stories_id': 1908931813,
  'tag': 'Clean Air Task Force',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 9600394},
 {'stories_id': 1908931813,
  'tag': 'federal General Services Administration',
  'tag_set': 'cliff_organizations',
  'tag_sets_id': 2388,
  'tags_id': 10271408}]